### Welcome to the COVID-19 Temperature Research Case Study



Can a correlation or correlations be found between weather conditions and positivity rates of COVID-19 within the United States? If so, such correlations may enable states or counties to better
implement rules and restrictions based on weather forecast.

In [1]:
## Importing neccesary libraries

import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['figure.figsize'] = (20,5)
import matplotlib.dates as md
'%matplotlib inline'
import seaborn as sns
import pandas as pd
import statsmodels.api as sm
import numpy as np

In [3]:
## Importing data file 
##dataframe_raw = pd.read_csv(r'C:\Users\Ana Salinas\Desktop\002_SpringBoard\Structured_Foundations\Capstone_2\COVID19_data.csv', header = 0, parse_dates =['date'], index_col=['date'])

dataframe_raw = pd.read_csv(r'C:\Users\Ana Salinas\Desktop\002_SpringBoard\Structured_Foundations\Capstone_2\COVID-19_HM.csv', header = 0, parse_dates =['Date'])


## Showing descriptive statistics for the data frame
d = dataframe_raw.describe()
display(d)

## Showing data types from all datafreame columns
dataframe_raw.info()

## Showing the fisrt 10 rows of the data frame
dataframe_raw.tail(10)

## Converting 3rd column into a type int

## dataframe_raw['Cumulative_Daily_Number_of_Cases']= dataframe_raw.Cumulative_Daily_Number_of_Cases.astype(int)

,CASES/DAY,CUMULATIVE_Daily_Cases,"NonCumulative_Cases/100,000","Cumulative_Cases/100,000",Max_Temp,Avg_Temp,Min_Temp
count,14416.000000,14416.000000,14416.000000,1.441600e+04,14416.000000,14416.000000,14416.000000
mean,567.250555,51829.806812,891.591842,5.433072e+04,53.772406,62.594756,71.708171
std,1211.963215,109368.291270,1747.173039,1.199699e+05,17.460839,17.955672,18.259283
min,-7757.000000,0.000000,0.000000,0.000000e+00,-24.000000,-11.000000,-2.000000
25%,6.000000,207.000000,17.000000,7.975000e+01,41.000000,50.000000,59.000000
50%,165.000000,9900.000000,322.000000,1.008550e+04,56.000000,66.000000,76.000000
75%,651.000000,55130.750000,1137.250000,6.061025e+04,68.000000,77.000000,86.000000
max,22276.000000,878346.000000,17847.000000,1.572162e+06,96.000000,658.000000,117.000000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14416 entries, 0 to 14415
Data columns (total 9 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   Date                         14416 non-null  datetime64[ns]
 1   State                        14416 non-null  object        
 2   CASES/DAY                    14416 non-null  int64         
 3   CUMULATIVE_Daily_Cases       14416 non-null  int64         
 4   NonCumulative_Cases/100,000  14416 non-null  int64         
 5   Cumulative_Cases/100,000     14416 non-null  int64         
 6   Max_Temp                     14416 non-null  int64         
 7   Avg_Temp                     14416 non-null  int64         
 8   Min_Temp                     14416 non-null  int64         
dtypes: datetime64[ns](1), int64(7), object(1)
memory usage: 1013.8+ KB


,Date,State,CASES/DAY,CUMULATIVE_Daily_Cases,"NonCumulative_Cases/100,000","Cumulative_Cases/100,000",Max_Temp,Avg_Temp,Min_Temp
14406,2020-10-09,Wyoming,243,7335,1267,75081,43,62,77
14407,2020-10-10,Wyoming,120,7455,1288,76369,43,58,76
14408,2020-10-11,Wyoming,156,7611,1315,77684,39,52,65
14409,2020-10-12,Wyoming,191,7802,1348,79033,42,54,66
14410,2020-10-13,Wyoming,162,7964,1376,80409,40,57,68
14411,2020-10-14,Wyoming,213,8177,1413,81821,34,53,66
14412,2020-10-15,Wyoming,198,8375,1447,83269,24,35,48
14413,2020-10-16,Wyoming,290,8665,1497,84766,20,47,67
14414,2020-10-17,Wyoming,151,8816,1523,86289,32,49,64
14415,2020-10-18,Wyoming,209,9025,1559,87848,27,30,35


### Data Cleansing:



The data was collected on a daily basis using copy/paste from a public website. The few missing
dates lacking temperature information were calculated using the average of the most recent
temperature known before and after the missing date. A total of 3 columns of 16,556 rows were
collected over all of the States and US regions. The collected temperatures were then matched to
every State in the main data set containing the daily cases. The main data set contains 643,740
rows pertaining to the daily cases recorded between 1/21/2020 - 10/18/2020. The only daily
temperature history not found were the data points from Guam and the Virgin Islands, for this
reason, these two US regions will be omitted in this case study.

Checking for missing values from the data file (none are found)

In [4]:
d.isna().any().any()
dataframe_raw.isna().any().any()


False

Checking for valid values (making sure there are no values that are too big/small)

In [5]:
## storing min a max values to print & check for boundaries 
max_of_max = max(dataframe_raw.Max_Temp)
min_of_max = min(dataframe_raw.Max_Temp)
max_of_min = max(dataframe_raw.Min_Temp)
min_of_min = min(dataframe_raw.Min_Temp)
max_of_avg = max(dataframe_raw.Avg_Temp)
min_of_avg = min(dataframe_raw.Avg_Temp)
median_of_avg = np.median(dataframe_raw.Avg_Temp)

print('Max value of Max Temperature = ', max_of_max, '\nMin value of Min Temperature = ', min_of_max,'\n')
print('Max value of Min Temperature = ', max_of_min, '\nMin value of Min Temperature = ', min_of_min,'\n')
print('Max value of Avg Temperature = ', max_of_avg, '\nMin value of Avg Temperature = ', min_of_avg,'\n')

print('\nAverage Temperature = ', median_of_avg)

print('\n\nThe values are within the expected boundaries.\nEnd of Data Cleansing!')

Max value of Max Temperature =  96 
Min value of Min Temperature =  -24 

Max value of Min Temperature =  117 
Min value of Min Temperature =  -2 

Max value of Avg Temperature =  658 
Min value of Avg Temperature =  -11 


Average Temperature =  66.0


The values are within the expected boundaries.
End of Data Cleansing!


### Main Issues

1. Finding the correlation, if any, between the temperature of each State and the number of daily cases. If
it does exist, the weather forecast can be used as a preventive measurement.

2. Finding the most accurate daily temperature per State to be assigned to the main data set containing
the daily cases.

3. Public sources are not always reliable and lack some of the temperature information for this study.

4. The study can not use the counties column due to the lack of complete public temperature information,
so the data collected was the one from the most populated city from each State.

5. Some States have a great variation of temperature on the same day.

6. Two regions (Guam and The Virgin Islands) had to be excluded due to the lack of daily temperature
information in the time period that is being covered.

### Insights

The data was separate in 3 categories. States with worst/hottest summers, States with worst/coldest
winters, and the remaining States make the last category. The purpose of this classification is to
demonstrate that people tend to ignore the stay at home orders where temperatures are extreme more so
than in the other States.

According to thillist.com the States with the most miserable winters are (ranked beginning from the worst):

        1.- Minnesota                 8.- Idaho
        2.- Michigan                  9.- Montana
        3.- Alaska                   10.- Massachusetts
        4.- North Dakota             11.- Iowa
        5.- Maine                    12.- Wyoming
        6.- South Dakota             13.- Illinois
        7.- Wisconsin

The States with the most miserable summers are:

        1.- Mississippi               8.- Florida
        2.- Louisiana                 9.- New Mexico
        3.- Arizona                  10.- South Carolina
        4.- Alabama                  11.- Kansas
        5.- Arkansas                 12.- Oklahoma
        6.- Georgia                  13.- Missouri
        7.- Texas
        
        


TO DO: Supporting visializations and conclusion!!